In [14]:
import numpy as np
from docplex.mp.model import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import time
%matplotlib inline

class Box:
    def __init__(self, id, x, y, z, w, d, h, bin):
        self.id = id
        self.x = x
        self.y = y
        self.z = z
        self.w = w
        self.h = h
        self.d = d
        self.bin = bin

    def to_dict(self):
        return {
            "id": self.id,
            "x": self.x,
            "y": self.y,
            "z": self.z,
            "width": self.w,
            "depth": self.d,
            "height": self.h,
            "bin": self.bin
        }
    
    def to_packer_box(self):
        return {
            "id": str(self.w) + str(self.d) + str(self.h) + "-" + str(self.id),
            "width": self.w,
            "depth": self.d,
            "height": self.h,
            "weight": 0.0,
            "maxLoad": 999.0,
            "availableInWarehouse": True,
            "onTop": False,
            "priority": 1,
            "incompatibilities": []
        }
def generateRotatedDuplices(boxes):
    newBoxes = []
    rotatedBoxes = []
    for box in boxes:
        newBoxes.append(box)
        newBoxes.append(Box(box.id, box.x, box.y, box.z, box.d, box.w, box.h, box.bin))
    return newBoxes

def parseInstance(instanceFile):
    W = 0
    D = 0
    H = 0
    boxes = []
    problemFile = open(instanceFile, 'r')
    for line in problemFile.readlines():
        commands = line.split(" ")
        args = commands[1].split(",")
        if commands[0] == "bin":
            W = int(args[0])
            D = int(args[1])
            H = int(args[2])
        if commands[0] == "box":
            box = Box(int(args[0]), 0, 0, 0, int(args[1]), int(args[2]), int(args[3]), 0)
            boxes.append(box);
    return (W, D, H, boxes)


#Utility functions (probably will be used in discretization)
def calculateOverlap(i, b1, k, b2, xk, yk):
    if b1 != b2: 
        return 0
    ximax = boxes[i].w
    yimax = boxes[i].d
    xkmax = xk + boxes[k].w
    ykmax = yk + boxes[k].d
    dx = min(ximax, xkmax) - max(0, xk)
    dy = min(yimax, ykmax) - max(0, yk)
    #value = 0 if dx<0 or dy<0 else dx*dy
    #if value != 0:
    #    print("Overlap {},{},{},{},{},{} = {}".format(i, b1, k, b2, xk, yk, value))
    return 0 if dx<0 or dy<0 else dx*dy

In [15]:
def getSolutionBox(i, boxes, solution):
    box = boxes[i]
    used_in = [b for b in B if solution.get_value(u[i][b]) == 1]
    if len(used_in) == 0:
        return None
    bin = used_in[0]
    return Box(
        box.id,
        int(round(solution.get_value(x[i]))),
        int(round(solution.get_value(y[i]))),
        int(round(solution.get_value(z[i]))),
        box.w,
        box.d,
        box.h,
        bin)

def cuboid_data(o, size=(1,1,1)):
    X = [[[0, 1, 0], [0, 0, 0], [1, 0, 0], [1, 1, 0]],
         [[0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 0, 0]],
         [[1, 0, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1]],
         [[0, 0, 1], [0, 0, 0], [0, 1, 0], [0, 1, 1]],
         [[0, 1, 0], [0, 1, 1], [1, 1, 1], [1, 1, 0]],
         [[0, 1, 1], [0, 0, 1], [1, 0, 1], [1, 1, 1]]]
    X = np.array(X).astype(float)
    for i in range(3):
        X[:,:,i] *= size[i]
    X += np.array(o)
    return X

def getCubeCollection(boxes,colors=None, **kwargs):
    if not isinstance(colors,(list,np.ndarray)): colors=["C0"]*len(boxes)
    g = []
    for box in boxes:
        g.append(cuboid_data((box["x"], box["y"], box["z"]), (box["width"], box["depth"], box["height"])))
    return Poly3DCollection(np.concatenate(g), facecolors=np.repeat(colors,6), **kwargs)

def plotBox(box, ax=None,**kwargs):
    if not isinstance(colors,(list,np.ndarray)): colors=["C0"]*len(positions)
    # Plotting a cube element at position pos
    if ax !=None:
        X, Y, Z = cuboid_data((box["x"], box["y"], box["z"]), (box["width"], box["depth"], box["height"]) )
        surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, **kwargs)
        surf.patch.set_edgecolor('black')
        ax.plot_wireframe(X, Y, Z, rstride=1, cstride=1, color="black", **kwargs)

colors = ["crimson","limegreen", "navy", "purple"]

In [16]:
## Check solution
def overlaps(xmin1, xmin2, xmax1, xmax2):
    return xmax1 > xmin2 and xmin1 < xmax2

def overlapArea(boxi, boxj):
    dx = (boxj.x - boxi.x)
    dy = (boxj.y - boxi.y)
    ximax = boxi.w
    yimax = boxi.d
    xkmax = dx + boxj.w
    ykmax = dy + boxj.d
    dx = min(ximax, xkmax) - max(0, dx)
    dy = min(yimax, ykmax) - max(0, dy)
    #value = 0 if dx<0 or dy<0 else dx*dy
    #if value != 0:
    #    print("Overlap {},{},{},{},{},{} = {}".format(i, b1, k, b2, xk, yk, value))
    return 0 if dx<0 or dy<0 else dx*dy

def checkSolution(boxes, solution):
    solutions = [getSolutionBox(i, boxes, solution) for i in I]
    nonNulls = [box for box in solutions if box != None]
    if len(nonNulls) != len(originalBoxes):
        return (False, "Some boxes weren't packed")
    for box in nonNulls:
        for other in nonNulls:
            if other != box and other.bin == box.bin:
                overlapX = overlaps(box.x, other.x, box.x + box.w, other.x + other.w)
                overlapY = overlaps(box.y, other.y, box.y + box.d, other.y + other.d)
                overlapZ = overlaps(box.z, other.z, box.z + box.h, other.z + other.h)
                if overlapX and overlapY and overlapZ:
                    return (False, "Box {} overlaps box {}".format(box.id, other.id))
    for box in nonNulls:
        supp_area = 0
        if box.z > beta_s:
            for other in nonNulls:
                if other != box and other.bin == box.bin and other.z + other.h <= box.z and box.z - (other.z + other.h) <= beta_s:
                    supp_area += overlapArea(box, other)
            if supp_area < alpha_s*box.w*box.d:
                return (False, "Box {} got area support {}".format(box.id, supp_area/(box.w*box.d)))
    return (True, "")

In [18]:
outDir = "tests/results/raw/model/solutions/"
testInstanceDir = "tests/instances/model/"

prevSolution = None
shouldReturn = False
with open("tests/results/raw/model/summary.csv", "w") as summaryFile:
    for instanceId in np.arange(1,20):
        instanceFile = testInstanceDir + "instance-" + str(instanceId) + ".test"
        print("Starting instance " + str(instanceId))
        mdl = Model(name="3DBPP")
        problem = parseInstance(instanceFile)
        W = problem[0]
        D = problem[1]
        H = problem[2]
        V = W*D*H
        DU = 10
        bins = 1
        originalBoxes = problem[3]
        boxes = generateRotatedDuplices(originalBoxes)
        I_O = np.arange(len(originalBoxes))
        I = np.arange(len(boxes))
        B = np.arange(bins)
        alpha_s = 0.7
        beta_s = 5

        maxDim = max(max(boxes[i].w, boxes[i].d) for i in I)
        I_R = np.arange(-int(np.floor(maxDim/DU)), int(np.floor(maxDim/DU)))

        # Problem Variables
        v = [mdl.integer_var(0, 1, "v_" + str(b)) for b in B]
        u = [[mdl.integer_var(0, 1, "u_" + str(i) + "_" + str(b)) for b in B] for i in I]
        x = [mdl.continuous_var(0, W-min(boxes[i].w, boxes[i].d), "x_" + str(i)) for i in I]
        y = [mdl.continuous_var(0, D-min(boxes[i].w, boxes[i].d), "y_" + str(i)) for i in I]
        z = [mdl.continuous_var(0, H-boxes[i].h, "z_" + str(i)) for i in I]
        xp = [[mdl.integer_var(0, 1, "xp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        yp = [[mdl.integer_var(0, 1, "yp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zp = [[mdl.integer_var(0, 1, "zp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zc = [[mdl.integer_var(0, 1, "zc_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zmax = [mdl.continuous_var(0, H, "zmax_" + str(b)) for b in B]
        s = [[mdl.integer_var(0, 1, "s_" + str(i) + "_" + str(k)) for k in I] for i in I]
        g = [mdl.integer_var(0, 1, "g_" + str(i)) for i in I]
        sp = [[[[[mdl.integer_var(0, 1, "sp_" + str(i) + "_" + str(k)  + "_" + str(b) + "_" + str(dx) + "_" + str(dy)) for dy in I_R] for dx in I_R] for b in B ] for k in I] for i in I]

        #Objective
        #mdl.minimize(mdl.sum((H*v[b] + zmax[b]) for b in B))                                                                                     #1
        mdl.minimize(mdl.sum(zmax[b] for b in B))
        
        #Constraints
        #[mdl.add_constraint(mdl.sum(u[i][b] for b in B) == 1) for i in I]
        [mdl.add_constraint(mdl.sum(u[i*2][b] for b in B) + mdl.sum(u[(i*2)+1][b] for b in B) == 1) for i in I_O]
        [mdl.add_constraint(u[i][b] <= v[b]) for b in B for i in I]

        [mdl.add_constraint(x[i] + boxes[i].w <= W) for i in I]
        [mdl.add_constraint(y[i] + boxes[i].d <= D) for i in I]
        [mdl.add_constraint(z[i] + boxes[i].h <= H) for i in I]

        [mdl.add_constraint(zmax[b] >= (z[i] + boxes[i].h) - H*(1-u[i][b])) for i in I for b in B]                                                             #9

        [mdl.add_constraint((x[i] + boxes[i].w) - x[j] <= W*(1-xp[i][j])) for i in I for j in I]
        [mdl.add_constraint(x[j] - (x[i] + boxes[i].w) + 1 <= W*xp[i][j]) for i in I for j in I]

        [mdl.add_constraint((y[i] + boxes[i].d) - y[j] <= D*(1-yp[i][j])) for i in I for j in I]
        [mdl.add_constraint(y[j] - (y[i] + boxes[i].d) + 1 <= D*yp[i][j]) for i in I for j in I]

        [mdl.add_constraint((z[i] + boxes[i].h) - z[j] <= H*(1-zp[i][j])) for i in I for j in I]
        [mdl.add_constraint(z[j] - (z[i] + boxes[i].h) + 1 <= H*zp[i][j]) for i in I for j in I]

        [mdl.add_constraint(v[b] >= v[c]) for b in B for c in B if c > b]

        [mdl.add_constraint(xp[i][j] + xp[j][i] +
                           yp[i][j] + yp[j][i] +
                           zp[i][j] + zp[j][i] >= u[i][b] + u[j][b] - 1) for i in I for j in I for b in B if not i == j]

        # Support constraint

        [mdl.add_constraint(z[j] - (z[i] + boxes[i].h) <= beta_s + H*(1-zc[i][j])) for i in I for j in I if i != j]
        [mdl.add_constraint(z[j] - (z[i] + boxes[i].h) >= -beta_s - H*(1-zc[i][j])) for i in I for j in I if i != j]
        [mdl.add_constraint(s[i][j] <= zp[i][j]) for i in I for j in I]
        [mdl.add_constraint(s[i][j] <= zc[i][j]) for i in I for j in I]
        [mdl.add_constraint(s[i][j] >= zp[i][j] + zc[i][j] - 2) for i in I for j in I]
        [mdl.add_constraint(mdl.sum(s[i][j] for j in I) <= mdl.sum(u[i][b] for b in B)) for i in I]
        [mdl.add_constraint(z[i] <= H*(1 - g[i])) for i in I]

        [mdl.add_constraint(mdl.sum(sp[i][j][b][dx][dy] for dx in I_R for dy in I_R for b in B if calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0) <= s[i][j]) for i in I for j in I]

        [mdl.add_constraint(x[j] - x[i] >= DU*dx - 2*W*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]
        [mdl.add_constraint(x[j] - x[i] <= DU*(dx + 1) + 2*W*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]
        [mdl.add_constraint(y[j] - y[i] >= DU*dy - 2*D*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]
        [mdl.add_constraint(y[j] - y[i] <= DU*(dy + 1) + 2*D*(1 - sp[i][j][b][dx][dy])) for dx in I_R for dy in I_R for b in B for i in I for j in I if i != j and calculateOverlap(i, b, j, b, dx*DU, dy*DU) != 0]

        [mdl.add_constraint(mdl.sum(calculateOverlap(j, b, i, b, dx*DU, dy*DU)*sp[j][i][b][dx][dy] for dx in I_R for dy in I_R for b in B for j in I if i != j and calculateOverlap(j, b, i, b, dx*DU, dy*DU) != 0) >= alpha_s*boxes[i].w*boxes[i].d - boxes[i].w*boxes[i].d*g[i]) for i in I]
        
        mdl.set_time_limit(3600)
            
        #Solve
        start = time.time()
        sol = mdl.solve(log_output=True)
        end = time.time()
        elapsedTime = (end - start)
        
        if (sol == None or sol.solve_details.gap > 4) and prevSolution != None and prevSolution.solve_details.gap < 0.5:
            start = time.time()
            mdl.add_mip_start(prevSolution)
            sol = mdl.solve(log_output=True)
            end = time.time()
            elapsedTime = (end - start)

        prevSolution = sol
        shouldReturn = prevSolution != None and prevSolution.solve_details.gap > 4
        print("Solution has a gap of " + str(prevSolution.solve_details.gap))
        solFile = outDir + "instance-" + str(instanceId) + ".json"
        prevSolution.export(solFile, format='json')
        
        validSolution = checkSolution(boxes, prevSolution)
        solutions = [getSolutionBox(i, boxes, prevSolution) for i in I]
        nonNulls = [box for box in solutions if box != None]
        summaryFile.write("{};{};{};{};{};{}\n".format(instanceId, elapsedTime, np.max([box.z + box.h for box in nonNulls]), np.sum([prevSolution.get_value(v[b]) for b in B]), validSolution[0], validSolution[1]))
        summaryFile.flush()
        
        #Plot solution
        if prevSolution and prevSolution.is_valid_solution():
            solutions = [getSolutionBox(i, boxes, prevSolution) for i in I]
            milpBoxes = [sol.to_dict() for sol in solutions if sol != None]
            openedBins = [b for b in B if prevSolution.get_value(v[b]) == 1]
            binPerRows = min(len(openedBins), 4)
            plotRows = int(len(openedBins)/binPerRows)
            fig = plt.figure(figsize=plt.figaspect(plotRows/binPerRows))
            idx = 0
            for bin in openedBins:
                idx += 1
                ax = fig.add_subplot(plotRows, binPerRows, idx, projection='3d')
                ax.set_title("Bin " + str(bin))
                ax.set_zlim(0, max([box["z"]+box["height"] for box in milpBoxes if box["bin"] == bin]))
                ax.set_ylim(0, D)
                ax.set_xlim(0, W)
                ax.add_collection3d(getCubeCollection([box for box in milpBoxes if box["bin"] == bin], colors=colors, edgecolor="k"))
            plt.savefig(outDir + "figures/" + "instance-" + str(instanceId) + ".pdf", bbox_inches='tight')
            plt.cla()
            plt.clf()
            
        if shouldReturn:
            break

Starting instance 1
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               3600
Tried aggregator 2 times.
MIP Presolve eliminated 942 rows and 815 columns.
MIP Presolve modified 1865 coefficients.
Aggregator did 1 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (2.32 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (2.43 ticks)
Parallel b&c, 16 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (2.43 ticks)
Solution has a gap of 0.0
Starting instance 2
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               3600
Tried aggregator 2 times.
MIP Presolve eliminated 2276 rows 

*     0+    0                          158.0000       85.0000            46.20%
      0     0  -1.00000e+75     0      158.0000       85.0000      114   46.20%
Detecting symmetries...
      0     2       85.0000     6      158.0000       85.0000      114   46.20%
Elapsed time = 1.05 sec. (1275.97 ticks, tree = 0.02 MB, solutions = 2)
    162    84       85.0000    19      158.0000       85.0000     1148   46.20%
*   393   234      integral     0       95.0000       85.0000     4891   10.53%
    395   248       85.0000    17       95.0000       85.0000     5100   10.53%
*   458   217      integral     0       85.0000       85.0000     5536    0.00%

Implied bound cuts applied:  38
Flow cuts applied:  4
Mixed integer rounding cuts applied:  11
Zero-half cuts applied:  7

Root node processing (before b&c):
  Real time             =    1.00 sec. (1269.95 ticks)
Parallel b&c, 16 threads:
  Real time             =    0.99 sec. (578.66 ticks)
  Sync time (average)   =    0.29 sec.
  Wait time

  25755  2496      150.0000    28      158.0000      149.0000   307100    5.70%
  26680  2539    infeasible            158.0000      149.0000   315811    5.70%
  27313  2442    infeasible            158.0000      149.0000   328927    5.70%
  27543  2289    infeasible            158.0000      149.0000   339150    5.70%
  28119  2021      150.0000    15      158.0000      149.0000   350705    5.70%
  29082  1936      153.0000    15      158.0000      150.0000   357530    5.06%
Elapsed time = 77.05 sec. (51279.25 ticks, tree = 17.24 MB, solutions = 5)
  30148  1857        cutoff            158.0000      150.0000   364868    5.06%
  30380  1519    infeasible            158.0000      150.0000   376175    5.06%
  30958  1102    infeasible            158.0000      153.0000   384233    3.16%
  32166   931      153.0000     8      158.0000      153.0000   391279    3.16%
  33270   807      153.0000     2      158.0000      153.0000   398564    3.16%
  33640   546    infeasible            158.00

  10724  2218    infeasible            158.0000       85.0000   143163   46.20%
Elapsed time = 49.75 sec. (34913.22 ticks, tree = 21.26 MB, solutions = 3)
  10758  2305       95.0000    16      158.0000       85.0000   149102   46.20%
  11057  2324      153.0000    23      158.0000       85.0000   151227   46.20%
  11742  2400      149.0000    32      158.0000       85.0000   153921   46.20%
  12514  2818       85.0000    19      158.0000       85.0000   170828   46.20%
  12865  3160    infeasible            158.0000       85.0000   185985   46.20%
  12929  3167       85.0000    25      158.0000       85.0000   188386   46.20%
  13321  3194       85.0000    38      158.0000       85.0000   192086   46.20%
  14016  3318      149.0000    27      158.0000       85.0000   200864   46.20%
  14647  3549      150.0000    21      158.0000       85.0000   210416   46.20%
  14721  3805      149.0000    56      158.0000       85.0000   221483   46.20%
Elapsed time = 67.14 sec. (44527.46 ticks, tr

  73527 18781        cutoff            158.0000       85.0000  1817871   46.20%
  74439 18657      153.0000    13      158.0000       85.0000  1851533   46.20%
  76352 18756       95.0000    42      158.0000       94.0000  1886625   40.51%
  79304 19000       95.0000    36      158.0000       95.0000  1919761   39.87%
  80638 19142       95.0000    31      158.0000       95.0000  1995781   39.87%
  80868 19199    infeasible            158.0000       95.0000  2048968   39.87%
  81762 19052      153.0000    38      158.0000       95.0000  2064169   39.87%
  83635 18973       99.0000    65      158.0000       95.0000  2075065   39.87%
Elapsed time = 379.27 sec. (219969.74 ticks, tree = 155.86 MB, solutions = 3)
  86270 18922       99.0000    26      158.0000       95.0000  2150932   39.87%
  87688 18943    infeasible            158.0000       95.0000  2204028   39.87%
  87908 18865    infeasible            158.0000       95.0000  2281666   39.87%
  88238 18603       99.0000    51      158

...

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

<Figure size 288x288 with 0 Axes>

In [25]:
import json 
outDir = "tests/results/raw/model/solutions/"
testInstanceDir = "tests/instances/model/"
for instanceId in np.arange(1,9):
    with open(outDir + "instance-" + str(instanceId) + ".json", "r") as solutionFile:
        jsonContent = solutionFile.readline()
        loadedSolution = json.loads(jsonContent)
        instanceFile = testInstanceDir + "instance-" + str(instanceId) + ".test"
        mdl = Model(name="3DBPP")
        problem = parseInstance(instanceFile)
        W = problem[0]
        D = problem[1]
        H = problem[2]
        V = W*D*H
        DU = 10
        bins = 1
        originalBoxes = problem[3]
        boxes = generateRotatedDuplices(originalBoxes)
        I_O = np.arange(len(originalBoxes))
        I = np.arange(len(boxes))
        B = np.arange(bins)
        alpha_s = 0.7
        beta_s = 5

        maxDim = max(max(boxes[i].w, boxes[i].d) for i in I)
        I_R = np.arange(-int(np.floor(maxDim/DU)), int(np.floor(maxDim/DU)))

        # Problem Variables
        v = [mdl.integer_var(0, 1, "v_" + str(b)) for b in B]
        u = [[mdl.integer_var(0, 1, "u_" + str(i) + "_" + str(b)) for b in B] for i in I]
        x = [mdl.continuous_var(0, W-min(boxes[i].w, boxes[i].d), "x_" + str(i)) for i in I]
        y = [mdl.continuous_var(0, D-min(boxes[i].w, boxes[i].d), "y_" + str(i)) for i in I]
        z = [mdl.continuous_var(0, H-boxes[i].h, "z_" + str(i)) for i in I]
        xp = [[mdl.integer_var(0, 1, "xp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        yp = [[mdl.integer_var(0, 1, "yp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zp = [[mdl.integer_var(0, 1, "zp_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zc = [[mdl.integer_var(0, 1, "zc_" + str(i) + "_" + str(k)) for k in I] for i in I]
        zmax = [mdl.continuous_var(0, H, "zmax_" + str(b)) for b in B]
        s = [[mdl.integer_var(0, 1, "s_" + str(i) + "_" + str(k)) for k in I] for i in I]
        g = [mdl.integer_var(0, 1, "g_" + str(i)) for i in I]
        sp = [[[[[mdl.integer_var(0, 1, "sp_" + str(i) + "_" + str(k)  + "_" + str(b) + "_" + str(dx) + "_" + str(dy)) for dy in I_R] for dx in I_R] for b in B ] for k in I] for i in I]

        class MyEncoder(json.JSONEncoder):
            def default(self, obj):
                if isinstance(obj, np.integer):
                    return int(obj)
                elif isinstance(obj, np.floating):
                    return float(obj)
                elif isinstance(obj, np.ndarray):
                    return obj.tolist()
                else:
                    return super(MyEncoder, self).default(obj)

        solution = mdl.new_solution()
        for var in loadedSolution["CPLEXSolution"]["variables"]:
            solution.add_var_value(var["name"], float(var["value"]))
        solutions = [getSolutionBox(i, boxes, solution) for i in I]
        nonNulls = [box for box in solutions if box != None]
        with open(outDir + "processed/instance-" + str(instanceId) + ".json", "w") as outFile:
            outFile.write(json.dumps([box.to_dict() for box in nonNulls], cls=MyEncoder))
        print(checkSolution(boxes, solution))

(True, '')
(True, '')
(True, '')
(True, '')
(True, '')
(True, '')
(True, '')
(False, 'Box 5 got area support 0.66796875')
